In [110]:
import pandas as pd
import scipy as sc
import numpy as np
import os


import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [111]:
from constants import guest_types
from constants import colors
from bootstrap import bootstrap
from bootstrap import dG_bootstrap
from bootstrap import dH_bootstrap
import plotting

In [112]:
from constants import experimental_deltaG
from constants import experimental_deltaH
from constants import systems

In [113]:
from summarize_statistics import write_statistics

# Calculate or load in the summary results

In [181]:
bgbg = pd.read_csv("results/bgbg_tip3p_by_orientation.csv")
bgbg_combined = pd.read_csv("results/bgbg_tip3p_combined.csv")

bg2bg2 = pd.read_csv("results/bg2bg2_tip3p_by_orientation.csv")
bg2bg2_combined = pd.read_csv("results/bg2bg2_tip3p_combined.csv")

smirnoff = pd.read_csv("results/smirnoff_by_orientation.csv")
smirnoff_combined = pd.read_csv("results/smirnoff_combined.csv")

experimental = pd.read_csv("results/experimental.csv")

In [ ]:
data_sets = [bgbg_combined, bg2bg2_combined, smirnoff_combined, experimental]
names = ["GAFF v1.7", "GAFF v2.1", "SMIRNOFF99Frosst", "Experimental"]

In [ ]:
from itertools import permutations

In [33]:
delta_G_r_squared = pd.DataFrame()
delta_G_rmse = pd.DataFrame()
delta_G_mse = pd.DataFrame()

delta_H_r_squared = pd.DataFrame()
delta_H_rmse = pd.DataFrame()
delta_H_mse = pd.DataFrame()

for index_i, i in enumerate(data_sets):
    for index_j, j in enumerate(data_sets):
                    
        # This seems the most reliable way of matching keys. Took a little while to figure out,
        # even if the data frames are ordered properly.
        df = i.merge(j, on=["System"], suffixes=("_i", "_j"))    
        
        results = bootstrap(df["Delta G_i"], df["G_SEM_i"], df["Delta G_j"], df["G_SEM_j"], cycles=1000)
        print(f"{names[index_i]} -- {names[index_j]} {results['mean']['R**2']}")


        delta_G_r_squared = delta_G_r_squared.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "R**2": results["mean"]["R**2"],
                "R**2 SEM": results["sem"]["R**2"]
            },
            ignore_index=True,
        )

        delta_G_rmse = delta_G_rmse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "RMSE": results["mean"]["RMSE"],
                "RMSE SEM": results["sem"]["RMSE"]
            },
            ignore_index=True,
        )

        delta_G_mse = delta_G_mse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "MSE": results["mean"]["MSE"],
                "MSE SEM": results["mean"]["MSE"]
            },
            ignore_index=True,
        )
        
        results = bootstrap(df["Delta H_i"], df["H_SEM_i"], df["Delta H_j"], df["H_SEM_j"], cycles=1000)
        print(f"{names[index_i]} -- {names[index_j]} {results['mean']['R**2']}")
        
        delta_H_r_squared = delta_H_r_squared.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "R**2": results["mean"]["R**2"],
                "R**2 SEM": results["sem"]["R**2"]
                
            },
            ignore_index=True,
        )

        delta_H_rmse = delta_H_rmse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "RMSE": results["mean"]["RMSE"],
                "RMSE SEM": results["sem"]["RMSE"]
            },
            ignore_index=True,
        )

        delta_H_mse = delta_H_mse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "MSE": results["mean"]["MSE"],
                "MSE SEM": results["sem"]["MSE"]
            },
            ignore_index=True,
        )


GAFF v1.7 -- GAFF v1.7 0.8640459088400202
GAFF v1.7 -- GAFF v1.7 0.8952935292070222
GAFF v1.7 -- GAFF v2.1 0.5443148855533857
GAFF v1.7 -- GAFF v2.1 0.6117083425882566
GAFF v1.7 -- SMIRNOFF99Frosst 0.5856835226983856
GAFF v1.7 -- SMIRNOFF99Frosst 0.6944846702335524
GAFF v1.7 -- Experimental 0.5370083937757651
GAFF v1.7 -- Experimental 0.3936289653837588
GAFF v2.1 -- GAFF v1.7 0.5377748892540649
GAFF v2.1 -- GAFF v1.7 0.6091330657408875
GAFF v2.1 -- GAFF v2.1 0.9574387696819072
GAFF v2.1 -- GAFF v2.1 0.974101682963238
GAFF v2.1 -- SMIRNOFF99Frosst 0.3711021040197053
GAFF v2.1 -- SMIRNOFF99Frosst 0.5979700428934158
GAFF v2.1 -- Experimental 0.8225389382298827
GAFF v2.1 -- Experimental 0.7507165075495107
SMIRNOFF99Frosst -- GAFF v1.7 0.5913937963590066
SMIRNOFF99Frosst -- GAFF v1.7 0.6967604030590443
SMIRNOFF99Frosst -- GAFF v2.1 0.36670847384122957
SMIRNOFF99Frosst -- GAFF v2.1 0.6040172606566303
SMIRNOFF99Frosst -- SMIRNOFF99Frosst 0.7451402000029268
SMIRNOFF99Frosst -- SMIRNOFF99Frosst

In [8]:
delta_G_r_squared_pivot = delta_G_r_squared.pivot(columns="y", index="x")
delta_G_rmse_pivot = delta_G_rmse.pivot(columns="y", index="x",)
delta_G_mse_pivot = delta_G_mse.pivot(columns="y", index="x",)

delta_H_r_squared_pivot = delta_H_r_squared.pivot(columns="y", index="x", )
delta_H_rmse_pivot = delta_H_rmse.pivot(columns="y", index="x",)
delta_H_mse_pivot = delta_H_mse.pivot(columns="y", index="x", )


In [9]:
pd.options.display.float_format = '{:,.2f}'.format
delta_G_r_squared_pivot

R**2                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.94      0.53      0.82             0.34   
GAFF v1.7                0.53      0.86      0.54             0.59   
GAFF v2.1                0.82      0.54      0.96             0.37   
SMIRNOFF99Frosst         0.34      0.59      0.37             0.74   

                     R**2 SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.08      0.10      0.08             0.11  
GAFF v1.7                0.10      0.04      0.10             0.11  
GAFF v2.1                0.08      0.10      0.01             0.11  
SMIRNOFF99Frosst         0.11      0.12      0.11             0.09

In [10]:
delta_G_rmse_pivot

RMSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.21      0.88      1.68             0.91   
GAFF v1.7                0.88      0.37      2.22             0.80   
GAFF v2.1                1.68      2.23      0.29             1.89   
SMIRNOFF99Frosst         0.91      0.79      1.90             0.46   

                     RMSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.16      0.09      0.09             0.10  
GAFF v1.7                0.09      0.04      0.12             0.12  
GAFF v2.1                0.09      0.12      0.03             0.13  
SMIRNOFF99Frosst         0.11      0.11      0.14             0.07

In [11]:
delta_G_mse_pivot

MSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.00      0.46     -1.56            -0.01   
GAFF v1.7               -0.46     -0.00     -2.01            -0.47   
GAFF v2.1                1.56      2.02     -0.00             1.54   
SMIRNOFF99Frosst         0.01      0.47     -1.54             0.00   

                      MSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.00      0.46     -1.56            -0.01  
GAFF v1.7               -0.46     -0.00     -2.01            -0.47  
GAFF v2.1                1.56      2.02     -0.00             1.54  
SMIRNOFF99Frosst         0.01      0.47     -1.54             0.00

In [12]:
delta_H_r_squared_pivot

R**2                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.99      0.39      0.75             0.44   
GAFF v1.7                0.39      0.89      0.61             0.70   
GAFF v2.1                0.75      0.61      0.97             0.60   
SMIRNOFF99Frosst         0.44      0.70      0.59             0.89   

                     R**2 SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.01      0.12      0.08             0.12  
GAFF v1.7                0.12      0.04      0.10             0.09  
GAFF v2.1                0.08      0.10      0.01             0.10  
SMIRNOFF99Frosst         0.11      0.09      0.10             0.04

In [13]:
delta_H_rmse_pivot

RMSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.15      2.55      2.21             1.85   
GAFF v1.7                2.54      0.71      3.88             1.65   
GAFF v2.1                2.20      3.88      0.44             2.96   
SMIRNOFF99Frosst         1.86      1.65      2.97             0.75   

                     RMSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.06      0.24      0.22             0.23  
GAFF v1.7                0.24      0.11      0.21             0.19  
GAFF v2.1                0.23      0.22      0.05             0.23  
SMIRNOFF99Frosst         0.22      0.17      0.23             0.12

In [14]:
delta_H_mse_pivot

MSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.00      1.85     -1.64             0.77   
GAFF v1.7               -1.84     -0.00     -3.48            -1.08   
GAFF v2.1                1.63      3.48      0.00             2.40   
SMIRNOFF99Frosst        -0.77      1.08     -2.41            -0.00   

                      MSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.03      0.27      0.22             0.27  
GAFF v1.7                0.27      0.11      0.26             0.20  
GAFF v2.1                0.23      0.27      0.07             0.27  
SMIRNOFF99Frosst         0.26      0.19      0.27             0.11

In [10]:
def pretty_print(results):
    print(f"{results['mean']['RMSE']:2.2f} ± {results['sem']['RMSE']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['MSE']:2.2f} ± {results['sem']['MSE']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['R**2']:2.2f} ± {results['sem']['R**2']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['slope']:2.2f} ± {results['sem']['slope']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['intercept']:2.2f} ± {results['sem']['intercept']:2.2f}", end="")
    print("\n")

# Binding free energy

(Create statistics files for re-use)

In [102]:
from multiprocessing import Pool

In [114]:
with Pool(12) as p:

    p.starmap(
        write_statistics,
        [
            (experimental, smirnoff_combined, "G", "experimental_smirnoff"),
            (experimental, bgbg_combined, "G", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "G", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "G", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "G", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "G", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "G", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "G", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "G", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bg2bg2_dG_statistics_overall.csv...
Skipping results/experimental_bgbg_dG_statistics_overall.csv...
Skipping results/experimental_bgbg_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dG_statistics_overall.csv...
Skipping results/experimental_bg2bg2_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bgbg_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_by_orientation_dG_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_d

## Enthalpy

In [115]:
with Pool(12) as p:

    p.starmap(
        write_statistics,
        [
            (experimental, smirnoff_combined, "H", "experimental_smirnoff"),
            (experimental, bgbg_combined, "H", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "H", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "H", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "H", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "H", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "H", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "H", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "H", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bgbg_dH_statistics_overall.csv...
Skipping results/experimental_bg2bg2_dH_statistics_overall.csv...
Skipping results/experimental_bgbg_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dH_statistics_overall.csv...
Skipping results/experimental_bgbg_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bg2bg2_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_smirnoff_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bgbg_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_dH_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_dH_statistics_al

# Entropy

In [182]:
bgbg["-TdS"] = bgbg["Delta G"] - bgbg["Delta H"]
bgbg["-TdS_SEM"] = np.sqrt(bgbg["G_SEM"]**2 + bgbg["H_SEM"]**2)

bg2bg2["-TdS"] = bg2bg2["Delta G"] - bg2bg2["Delta H"]
bg2bg2["-TdS_SEM"] = np.sqrt(bg2bg2["G_SEM"]**2 + bg2bg2["H_SEM"]**2)

smirnoff["-TdS"] = smirnoff["Delta G"] - smirnoff["Delta H"]
smirnoff["-TdS_SEM"] = np.sqrt(smirnoff["G_SEM"]**2 + smirnoff["H_SEM"]**2)


smirnoff_combined["-TdS"] = smirnoff_combined["Delta G"] - smirnoff_combined["Delta H"]
smirnoff_combined["-TdS_SEM"] = np.sqrt(smirnoff_combined["G_SEM"]**2 + smirnoff_combined["H_SEM"]**2)

bgbg_combined["-TdS"] = bgbg_combined["Delta G"] - bgbg_combined["Delta H"]
bgbg_combined["-TdS_SEM"] = np.sqrt(bgbg_combined["G_SEM"]**2 + bgbg_combined["H_SEM"]**2)

bg2bg2_combined["-TdS"] = bg2bg2_combined["Delta G"] - bg2bg2_combined["Delta H"]
bg2bg2_combined["-TdS_SEM"] = np.sqrt(bg2bg2_combined["G_SEM"]**2 + bg2bg2_combined["H_SEM"]**2)

experimental["-TdS"] = experimental["Delta G"] - experimental["Delta H"]
experimental["-TdS_SEM"] = np.sqrt(experimental["G_SEM"]**2 + experimental["H_SEM"]**2)

In [93]:
from summarize_statistics import write_entropy_statistics

In [120]:
with Pool(12) as p:

    p.starmap(
        write_entropy_statistics,
        [
            (experimental, smirnoff_combined, "-TdS", "experimental_smirnoff"),
            (experimental, bgbg_combined, "-TdS", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "-TdS", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "-TdS", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "-TdS", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "-TdS", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "-TdS", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "-TdS", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "-TdS", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bg2bg2_-TdS_statistics_overall.csv...
Skipping results/experimental_bgbg_-TdS_statistics_overall.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_overall.csv...
Skipping results/experimental_bgbg_-TdS_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_smirnoff_-TdS_statistics_overall.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/bgbg_smirnoff_-TdS_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_aliphatic_carboxylates.csv...
Skipping results/

In [125]:
from make_statistics_table import table, entropy_table

In [122]:
table(thermodynamic_quantity="G")

SMIRNOFF99Frosst	0.91	0.11	-0.01	0.14	0.34	0.11	0.49	0.12	-1.55	0.38	

GAFF v1.7	0.88	0.09	0.46	0.12	0.54	0.10	0.69	0.11	-0.48	0.35	

GAFF v2.1	1.68	0.09	-1.56	0.10	0.82	0.08	1.19	0.09	-1.00	0.28	



In [123]:
table(thermodynamic_quantity="H")

SMIRNOFF99Frosst	1.85	0.23	0.77	0.26	0.44	0.12	0.85	0.17	0.41	0.52	

GAFF v1.7	2.54	0.24	1.84	0.27	0.39	0.12	0.80	0.18	1.36	0.60	

GAFF v2.1	2.21	0.23	-1.64	0.23	0.75	0.08	1.38	0.12	-0.69	0.40	



In [126]:
entropy_table(thermodynamic_quantity="-TdS")

SMIRNOFF99Frosst	1.90	0.21	-0.78	0.27	0.40	0.13	0.90	0.20	-0.83	0.25	

GAFF v1.7	2.21	0.24	-1.38	0.27	0.43	0.13	0.96	0.21	-1.41	0.24	

GAFF v2.1	1.47	0.24	0.08	0.23	0.60	0.14	1.14	0.18	0.15	0.21	



In [139]:
df = pd.merge(experimental, smirnoff_combined, on=["System"], 
              suffixes=).merge(bgbg_combined, on=["System"])   


In [183]:
experimental = experimental.sort_values(by="System").reset_index(drop=True)
smirnoff_combined = smirnoff_combined.sort_values(by="System").reset_index(drop=True)
bgbg_combined = bgbg_combined.sort_values(by="System").reset_index(drop=True)
bg2bg2_combined = bg2bg2_combined.sort_values(by="System").reset_index(drop=True)

experimental = experimental.add_suffix('_experimental')
smirnoff_combined = smirnoff_combined.add_suffix('_smirnoff')
bgbg_combined = bgbg_combined.add_suffix('_bgbg')
bg2bg2_combined = bg2bg2_combined.add_suffix('_bg2bg2')


In [186]:
experimental.tail()

,Unnamed: 0_experimental,System_experimental,Delta G_experimental,G_SEM_experimental,Delta H_experimental,H_SEM_experimental,-TdS_experimental,-TdS_SEM_experimental
38,12,b-oam,-3.59,0.12,-0.48,0.03,-3.11,0.123693
39,41,b-pb3,-3.52,0.01,-2.25,0.01,-1.27,0.014142
40,42,b-pb4,-3.60,0.02,-2.82,0.01,-0.78,0.022361
41,36,b-pha,-1.70,0.05,-1.79,0.11,0.09,0.120830
42,32,b-pnt,-1.27,0.32,1.89,0.53,-3.16,0.619112


In [185]:
bg2bg2_combined.tail()

,Unnamed: 0_bg2bg2,Delta G_bg2bg2,Delta H_bg2bg2,G_SEM_bg2bg2,H_SEM_bg2bg2,System_bg2bg2,Type_bg2bg2,-TdS_bg2bg2,-TdS_SEM_bg2bg2
38,38,-5.245263,-0.518012,0.233721,0.338562,b-oam,aliphatic_ammoniums,-4.727251,0.411399
39,39,-4.583314,-2.242086,0.171185,0.297012,b-pb3,aliphatic_carboxylates,-2.341228,0.342812
40,40,-4.713973,-3.700640,0.227091,0.287580,b-pb4,aliphatic_carboxylates,-1.013333,0.366432
41,41,-3.979595,-3.454893,0.193098,0.358932,b-pha,aliphatic_carboxylates,-0.524702,0.407577
42,42,-2.003901,0.400442,0.164106,0.313086,b-pnt,aliphatic_carboxylates,-2.404342,0.353488


In [187]:
data = [experimental, smirnoff_combined, bgbg_combined, bg2bg2_combined]
df = data[0].join(data[1:])

In [191]:
df[["System_experimental",
   "Delta G_experimental",
   "G_SEM_experimental",
   "Delta G_smirnoff",
   "G_SEM_smirnoff",
   "Delta G_bgbg",
   "G_SEM_bgbg",
   "Delta G_bg2bg2",
   "G_SEM_bg2bg2"]]

,System_experimental,Delta G_experimental,G_SEM_experimental,Delta G_smirnoff,G_SEM_smirnoff,Delta G_bgbg,G_SEM_bgbg,Delta G_bg2bg2,G_SEM_bg2bg2
0,a-bam,-1.58,0.02,-3.246889,0.444726,-0.823798,0.206395,-2.934819,0.233797
1,a-but,-1.51,0.04,-1.485230,0.267585,-1.085933,0.204793,-3.139644,0.219072
2,a-cbu,-2.02,0.02,-1.332678,0.186850,-0.886669,0.215073,-3.730256,0.207096
3,a-chp,-2.51,0.06,-2.378949,0.281034,-1.693563,0.243640,-4.108140,0.227920
4,a-coc,-3.23,1.14,-1.780491,0.292164,-1.861277,0.243723,-3.351319,0.236707
5,a-cpe,-2.13,0.02,-1.593394,0.251643,-1.498724,0.291323,-3.791762,0.223456
6,a-ham,-3.53,0.00,-3.432186,0.296499,-3.015662,0.193813,-5.993404,0.171967
7,a-hep,-3.99,0.01,-3.947333,0.206577,-3.925517,0.200996,-6.233197,0.174546
8,a-hex,-3.38,0.01,-2.704837,0.212787,-2.923050,0.207345,-5.267280,0.204057
9,a-hp6,-3.60,0.00,-3.317278,0.225667,-3.368472,0.175419,-5.405712,0.178810


In [194]:
df[["System_experimental",
   "Delta H_experimental",
   "H_SEM_experimental",
   "Delta H_smirnoff",
   "H_SEM_smirnoff",
   "Delta H_bgbg",
   "H_SEM_bgbg",
   "Delta H_bg2bg2",
   "H_SEM_bg2bg2"]]

,System_experimental,Delta H_experimental,H_SEM_experimental,Delta H_smirnoff,H_SEM_smirnoff,Delta H_bgbg,H_SEM_bgbg,Delta H_bg2bg2,H_SEM_bg2bg2
0,a-bam,-2.17,0.05,-0.428111,0.280564,-0.843253,0.592789,-3.049590,0.382428
1,a-but,-2.53,0.12,-0.758697,0.593325,-1.075870,0.369402,-4.911070,0.415653
2,a-cbu,-2.75,0.05,-2.079338,0.214027,-0.714452,0.492323,-4.941228,0.292223
3,a-chp,-2.99,0.23,-3.419180,0.389139,-2.326133,0.279547,-5.267667,0.345117
4,a-coc,-0.93,0.32,-3.803446,0.446404,-2.927497,0.316008,-6.172950,0.324944
5,a-cpe,-2.74,0.02,-1.925657,0.297781,-1.057938,0.417167,-4.858278,0.293909
6,a-ham,-4.19,0.02,-4.018099,0.327923,-2.333386,0.302466,-6.909934,0.289129
7,a-hep,-4.19,0.09,-4.719162,0.327539,-4.046608,0.355618,-8.682060,0.236540
8,a-hex,-3.40,0.02,-4.326430,0.298402,-2.949684,0.299115,-7.433955,0.307058
9,a-hp6,-4.48,0.02,-4.858886,0.305813,-3.733573,0.210896,-8.241340,0.311997


In [195]:
df[["System_experimental",
   "-TdS_experimental",
   "-TdS_SEM_experimental",
   "-TdS_smirnoff",
   "-TdS_SEM_smirnoff",
   "-TdS_bgbg",
   "-TdS_SEM_bgbg",
   "-TdS_bg2bg2",
   "-TdS_SEM_bg2bg2"]]

,System_experimental,-TdS_experimental,-TdS_SEM_experimental,-TdS_smirnoff,-TdS_SEM_smirnoff,-TdS_bgbg,-TdS_SEM_bgbg,-TdS_bg2bg2,-TdS_SEM_bg2bg2
0,a-bam,0.59,0.053852,-2.818778,0.525830,0.019455,0.627692,0.114771,0.448232
1,a-but,1.02,0.126491,-0.726534,0.650874,-0.010063,0.422372,1.771427,0.469851
2,a-cbu,0.73,0.053852,0.746660,0.284114,-0.172217,0.537251,1.210972,0.358167
3,a-chp,0.48,0.237697,1.040231,0.480009,0.632569,0.370819,1.159527,0.413585
4,a-coc,-2.30,1.184061,2.022955,0.533513,1.066220,0.399076,2.821631,0.402018
5,a-cpe,0.61,0.028284,0.332263,0.389869,-0.440786,0.508819,1.066516,0.369209
6,a-ham,0.66,0.020000,0.585913,0.442092,-0.682276,0.359234,0.916530,0.336405
7,a-hep,0.20,0.090554,0.771829,0.387241,0.121092,0.408490,2.448863,0.293968
8,a-hex,0.02,0.022361,1.621594,0.366500,0.026633,0.363953,2.166675,0.368679
9,a-hp6,0.88,0.020000,1.541608,0.380062,0.365101,0.274315,2.835628,0.359604
